<a href="https://colab.research.google.com/github/geryabra/SMP/blob/main/Monthly_Bids_and_Availability_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Excel Monthly Automation – Colab (Restart & Run All)
מחברת נקייה ומעודכנת:
- העלאת קבצים ידנית
- זיהוי קשיח של bid_rangesALT34 ו-יצרן
- טרנספורמציות יציבות (transform_availability_v2)
- ייצוא שני קבצי XLSX נפרדים ל-outputs

**הוראות:** Runtime → Restart runtime → Run all


## Cell 0 — Upload files

In [9]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))

Saving Availability.xlsx to Availability.xlsx
Saving Book2.xlsx to Book2.xlsx
Saving אלון תבור 01.2025.xlsx to אלון תבור 01.2025.xlsx
Saving אלון תבור 02.2025.xlsx to אלון תבור 02.2025.xlsx
Uploaded: ['Availability.xlsx', 'Book2.xlsx', 'אלון תבור 01.2025.xlsx', 'אלון תבור 02.2025.xlsx']


## Cell 1 — Install & Imports

In [10]:
!pip -q install pandas openpyxl
import os, re, glob
from datetime import datetime
import pandas as pd
import numpy as np
import openpyxl
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

## Cell 2 — Paths

In [40]:
from google.colab import drive
drive.mount('/content/drive')

INPUT_DIR = "/content"
OUTPUT_DIR = os.path.join("/content/drive/My Drive", "Colab Outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)
print("INPUT_DIR:", INPUT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INPUT_DIR: /content
OUTPUT_DIR: /content/drive/My Drive/Colab Outputs


## Cell 3 — Sheet Detection

In [12]:
SHEET_DETECTION = {
    "bid_range": [r"^bid[_\s-]*ranges?", r"^bidrange"],
    "availability": [r"יצרן", r"availability"]
}
def normalize_sheet_name(s: str) -> str:
    return re.sub(r"\s+", " ", str(s or "").strip())
def detect_logical_sheet(sheet_name: str):
    s = normalize_sheet_name(sheet_name).lower()
    for logical, patterns in SHEET_DETECTION.items():
        for pat in patterns:
            if re.search(pat, s, flags=re.IGNORECASE):
                return logical
    return None

## Cell 4 — Helpers

In [22]:
def safe_list_excels(input_dir):
    return sorted(glob.glob(os.path.join(input_dir, "*.xlsx"))) + sorted(glob.glob(os.path.join(input_dir, "*.xlsm")))
def get_sheet_names(excel_path):
    wb = openpyxl.load_workbook(excel_path, read_only=True, data_only=True)
    return wb.sheetnames
def coerce_date_time_columns(df, date_col="Date", time_col="Time"):
    out = df.copy()
    out.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in out.columns]
    def find_col(candidates):
        norm = {str(c).strip().lower(): c for c in out.columns}
        for cand in candidates:
            key = cand.strip().lower()
            if key in norm:
                return norm[key]
        return None
    date_candidates = [date_col, "date", "יום", "day", "תאריך"]
    time_candidates = [time_col, "time", "שעה", "hour", "hh:mm", "hours"]
    real_date = find_col(date_candidates)
    real_time = find_col(time_candidates)
    if real_date is None or real_time is None:
        raise ValueError(f"Date/Time not found. Columns: {list(out.columns)[:40]}")
    if real_date != date_col:
        out = out.rename(columns={real_date: date_col})
    if real_time != time_col:
        out = out.rename(columns={real_time: time_col})
    out[date_col] = pd.to_datetime(out[date_col], errors="coerce").dt.date
    t = out[time_col]
    parsed = pd.to_datetime(t, errors="coerce")
    if parsed.isna().mean() > 0.5:
        parsed = pd.to_datetime(t.astype(str), errors="coerce")
    out[time_col] = parsed.dt.time
    return out
def make_unique_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cols = [str(c) for c in df.columns]
    seen = {}
    new_cols = []
    for c in cols:
        c_clean = re.sub(r"\s+", " ", c).strip()
        if c_clean not in seen:
            seen[c_clean] = 0
            new_cols.append(c_clean)
        else:
            seen[c_clean] += 1
            new_cols.append(f"{c_clean}__dup{seen[c_clean]}")
    df.columns = new_cols
    return df


## Cell 5 — Transform bid_range

In [17]:
def transform_bid_range(excel_path, sheet_name):
    raw = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

    header_date_row = None
    for r in range(0, min(40, len(raw))):
        row_vals = raw.iloc[r].astype(str).str.lower().tolist()
        if "date" in row_vals and "time" in row_vals:
            header_date_row = r
            break
    if header_date_row is None:
        raise ValueError("Date/Time header not found")

    header_name_row = max(0, header_date_row - 1)
    data_start_row = header_date_row + 1

    cols = []
    for c in range(raw.shape[1]):
        v_date = raw.iat[header_date_row, c]
        v_name = raw.iat[header_name_row, c]
        if isinstance(v_date, str) and v_date.strip() in ["Date", "Time"]:
            cols.append(v_date.strip())
        elif isinstance(v_name, str) and v_name and str(v_name).strip() != "nan":
            cols.append(str(v_name).strip())
        else:
            cols.append(f"col_{c}")

    df = raw.iloc[data_start_row:].copy()
    df.columns = cols
    df = df.dropna(axis=1, how="all")
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]

    def find_contains(regex):
        pat = re.compile(regex, flags=re.IGNORECASE)
        for c in df.columns:
            if pat.search(c):
                return c
        return None

    resolved = ["Date", "Time"]
    for i in range(1, 11):
        low = find_contains(rf"OfferDA.*Range.*Low.*{i}")
        up  = find_contains(rf"OfferDA.*Range.*Up.*{i}")
        pr  = find_contains(rf"OfferDA.*Price.*DA\s*{i}")
        if not all([low, up, pr]):
            raise ValueError(f"Missing OfferDA columns for i={i}")
        resolved += [low, up, pr]

    out = df[resolved].copy()

    # ✅ FIX: start mapping from index 2 (not 1), so Time stays Time
    rename = {"Date": "Date", "Time": "Time"}
    for i in range(1, 11):
        base = 2 + (i - 1) * 3
        rename[resolved[base + 0]] = f"BidSell Low Limit{i}"
        rename[resolved[base + 1]] = f"BidSell Up Limit{i}"
        rename[resolved[base + 2]] = f"BidSell DA{i}"

    out = out.rename(columns=rename)

    # now Date+Time exist correctly
    out = coerce_date_time_columns(out, "Date", "Time")

    for i in range(1, 11):
        for c in [f"BidSell Low Limit{i}", f"BidSell Up Limit{i}", f"BidSell DA{i}"]:
            out[c] = pd.to_numeric(out[c], errors="coerce")

    out["ממוצע בידים"] = out[[f"BidSell DA{i}" for i in range(1, 11)]].mean(axis=1, skipna=True)
    return out


## Cell 6 — Transform Availability (v2)

In [35]:
def transform_availability_v2(excel_path, sheet_name):
    raw = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

    # 1) למצוא את שורת הכותרת שבה מופיעים יום ושעה (בלי תלות בעמודה J)
    header_row = None
    for r in range(0, min(120, len(raw))):
        row = raw.iloc[r].astype(str).str.strip().tolist()
        if ("יום" in row) and ("שעה" in row):
            header_row = r
            break

    if header_row is None:
        raise ValueError(f"לא נמצאה שורת כותרות עם 'יום'/'שעה' בגיליון {sheet_name}")

    # 2) נתונים: מתחילים שורה אחרי הכותרות
    df = raw.iloc[header_row + 1:].copy()

    # 3) לקחת A,B,J לפי אינדקסים 0,1,9
    if df.shape[1] < 10:
        raise ValueError(f"גיליון {sheet_name} מכיל פחות מ-10 עמודות ולכן לא ניתן לקחת A,B,J. cols={df.shape[1]}")

    out = df.iloc[:, [0, 1, 9]].copy()
    out.columns = ["Date", "Time", "Availability"]

    # 4) ניקוי שורות ריקות
    out = out.dropna(how="all")

    # 5) המרות
    out = coerce_date_time_columns(out, "Date", "Time")
    out["Availability"] = pd.to_numeric(out["Availability"], errors="coerce")

    return out


## Cell 7 — Run pipeline

In [36]:
# Cell 7 — Run pipeline (FINAL, updated for Availability = 3 columns only)

# 1) קבצי חודש בלבד (לא תבניות)
excel_files = [
    f for f in safe_list_excels(INPUT_DIR)
    if os.path.basename(f) not in ["Book2.xlsx", "Availability.xlsx"]
]

print("Found files:", [os.path.basename(x) for x in excel_files])

if not excel_files:
    raise ValueError("לא נמצאו קבצי חודש לעיבוד")

book2_parts = []
avail_parts = []

# 2) מעבר על קבצים וגיליונות
for path in excel_files:
    print("\nFile:", os.path.basename(path))
    sheets = get_sheet_names(path)

    for s in sheets:
        kind = detect_logical_sheet(s)
        print(" ", repr(s), "=>", kind)

        # ---------- BID RANGE ----------
        if kind == "bid_range":
            dfb = transform_bid_range(path, s)
            dfb = make_unique_columns(dfb)

            dfb["SourceFile"] = os.path.basename(path)
            dfb["SourceSheet"] = s

            book2_parts.append(dfb)

        # ---------- AVAILABILITY (3 columns only) ----------
        elif kind == "availability":
            dfa = transform_availability_v2(path, s)
            dfa = make_unique_columns(dfa)

            # אופציונלי: אם אתה רוצה רק 3 עמודות נטו (בלי Source*)
            # dfa = dfa[["Date", "Time", "Availability"]]

            dfa["SourceFile"] = os.path.basename(path)
            dfa["SourceSheet"] = s

            avail_parts.append(dfa)

# 3) ניקוי כפילויות אחרון לפני concat (חסין InvalidIndexError)
def force_unique_cols(df):
    df = make_unique_columns(df)
    if df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()].copy()
    return df

book2_parts = [force_unique_cols(d) for d in book2_parts]
avail_parts = [force_unique_cols(d) for d in avail_parts]

# 4) איחוד
book2_out_df = pd.concat(book2_parts, ignore_index=True) if book2_parts else pd.DataFrame()
avail_out_df = pd.concat(avail_parts, ignore_index=True) if avail_parts else pd.DataFrame()

print("\nbook2_out_df shape:", book2_out_df.shape)
print("avail_out_df shape:", avail_out_df.shape)

if book2_out_df.empty:
    raise ValueError("book2_out_df ריק – לא נאספו נתוני bid_range")

if avail_out_df.empty:
    raise ValueError("avail_out_df ריק – לא נאספו נתוני availability")

display(book2_out_df.head(3))
display(avail_out_df.head(3))


Found files: ['אלון תבור 01.2025.xlsx', 'אלון תבור 02.2025.xlsx']

File: אלון תבור 01.2025.xlsx
  'יצרן ALTV34' => availability


/tmp/ipython-input-3911510318.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(t.astype(str), errors="coerce")


  'bid_rangesALT34' => bid_range


/tmp/ipython-input-3911510318.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(t.astype(str), errors="coerce")



File: אלון תבור 02.2025.xlsx
  'bid_rangesALT34' => bid_range


/tmp/ipython-input-3911510318.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(t.astype(str), errors="coerce")


  'יצרן ALTV34' => availability

book2_out_df shape: (2976, 35)
avail_out_df shape: (2978, 5)


/tmp/ipython-input-3911510318.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(t.astype(str), errors="coerce")


,Date,Time,BidSell Low Limit1,BidSell Up Limit1,BidSell DA1,BidSell Low Limit2,BidSell Up Limit2,BidSell DA2,BidSell Low Limit3,BidSell Up Limit3,BidSell DA3,BidSell Low Limit4,BidSell Up Limit4,BidSell DA4,BidSell Low Limit5,BidSell Up Limit5,BidSell DA5,BidSell Low Limit6,BidSell Up Limit6,BidSell DA6,BidSell Low Limit7,BidSell Up Limit7,BidSell DA7,BidSell Low Limit8,BidSell Up Limit8,BidSell DA8,BidSell Low Limit9,BidSell Up Limit9,BidSell DA9,BidSell Low Limit10,BidSell Up Limit10,BidSell DA10,ממוצע בידים,SourceFile,SourceSheet
0,2025-01-01,00:00:00,85.005,91.005,85.11,91.01,97.01,85.11,97.015,103.015,85.11,103.02,109.02,85.11,109.025,115.025,85.11,115.03,121.03,88.22,121.035,127.035,88.22,127.04,133.04,88.22,133.045,139.045,88.22,139.05,145.0,88.22,86.665,אלון תבור 01.2025.xlsx,bid_rangesALT34
1,2025-01-01,00:30:00,85.005,91.005,85.11,91.01,97.01,85.11,97.015,103.015,85.11,103.02,109.02,85.11,109.025,115.025,85.11,115.03,121.03,88.22,121.035,127.035,88.22,127.04,133.04,88.22,133.045,139.045,88.22,139.05,145.0,88.22,86.665,אלון תבור 01.2025.xlsx,bid_rangesALT34
2,2025-01-01,01:00:00,85.005,91.005,85.11,91.01,97.01,85.11,97.015,103.015,85.11,103.02,109.02,85.11,109.025,115.025,85.11,115.03,121.03,88.22,121.035,127.035,88.22,127.04,133.04,88.22,133.045,139.045,88.22,139.05,145.0,88.22,86.665,אלון תבור 01.2025.xlsx,bid_rangesALT34


,Date,Time,Availability,SourceFile,SourceSheet
0,1970-01-01,NaT,NaN,אלון תבור 01.2025.xlsx,יצרן ALTV34
1,2025-01-01,00:00:00,85.504849,אלון תבור 01.2025.xlsx,יצרן ALTV34
2,2025-01-01,00:30:00,26.678397,אלון תבור 01.2025.xlsx,יצרן ALTV34


## Cell 8 — Export

In [44]:
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
book2_path = os.path.join(OUTPUT_DIR, f"Book2_{stamp}.xlsx")
avail_path = os.path.join(OUTPUT_DIR, f"Availability_{stamp}.xlsx")
book2_out_df.to_excel(book2_path, index=False)
avail_out_df.to_excel(avail_path, index=False)
print("Saved:", book2_path, avail_path)

Saved: /content/drive/My Drive/Colab Outputs/Book2_20251226_110947.xlsx /content/drive/My Drive/Colab Outputs/Availability_20251226_110947.xlsx


In [46]:
import os

print(f"Listing contents of {OUTPUT_DIR}:")
for root, dirs, files in os.walk(OUTPUT_DIR):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name) + '/')


Listing contents of /content/drive/My Drive/Colab Outputs:
/content/drive/My Drive/Colab Outputs/Book2_20251226_110947.xlsx
/content/drive/My Drive/Colab Outputs/Availability_20251226_110947.xlsx
